# Materials

In [ ]:
%matplotlib inline
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
c_fck = np.array([20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90])
c_epsc2 = np.array( [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.3, 2.4 , 2.5, 2.6])
c_epscu2 = np.array([3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 2.9, 2.7 , 2.6, 2.6])
c_n = np.array(     [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.6, 1.45, 1.4, 1.4])

In [ ]:
import eurocodepy as ec
database = ec.get_database()
concs = database["Eurocodes"]["Materials"]["Concrete"]
reinfs = database["Eurocodes"]["Materials"]["Reinforcement"]
prestr = database["Eurocodes"]["Materials"]["Prestress"]
ec2conc = pd.DataFrame.from_dict(concs["Classes"])
ec2reinf = pd.DataFrame.from_dict(reinfs["Classes"])

In [ ]:
ec2conc

In [ ]:
ec2reinf

## Concrete diagrams

In [ ]:
n = 20
eps = np.linspace(0, 1, n)
for i in range(11):
    stres = 1.0-np.power(1.0-np.linspace(0, 1, n), c_n[i])
    eps2 = np.concatenate((eps*c_epsc2[i] , np.linspace(c_epsc2[i],c_epscu2[i],3)))
    stres2 = np.concatenate((stres, np.ones(3)))*c_fck[i]
    plt.plot(eps2, stres2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set_theme(style="darkgrid")

df = pd.DataFrame(columns=['epsilon', 'sigma', 'class'])
n = 20

for i, cclass in enumerate(ec2conc.columns):
    stres = 1-np.power(1-np.linspace(0, 1, n), c_n[i])
    eps = np.linspace(0, 1, n)
    df2 = pd.DataFrame(columns=['epsilon', 'sigma', 'class'])
    df2['epsilon'] = np.concatenate((eps*c_epsc2[i] , np.linspace(c_epsc2[i],c_epscu2[i],3)))
    df2['sigma'] = np.concatenate((stres, np.ones(3)))*c_fck[i]
    df2['class'] = [cclass for j in range(len(eps2))]
    df = df.append(df2, ignore_index=True)

sns.set_context("notebook", font_scale=1)
plt.figure(figsize=(8,5))
sns.lineplot(x="epsilon", y="sigma", hue="class", data=df)
plt.xlabel("Strain (x1000)")
plt.ylabel("Stress (MPa)")
plt.legend(bbox_to_anchor=(1.01, 1),
           borderaxespad=0)
plt.tight_layout()
#plt.savefig("place_legend_outside_plot_Seaborn_scatterplot.png", format='png',dpi=150)
plt.show()

## Evolução das propriedades no tempo

In [ ]:
# cement type parameter s
# s = 0.20, endurecimento rápido R: CEM42,5R, CEM52,5N e CEM52,5R
# s = 0.25, endurecimento normal N: CEM32,5R, CEM42,5N
# s = 0.38, endurecimento lento S: CEM32,5N
s = [0.2, 0.25, 0.38]
n = 50
t = np.logspace(0.01, 1.9, n)
# Resistência à compressão
for i in range(3):
    betha_cc = np.exp(s[i]*(1.0-np.sqrt(28.0/t)))
    plt.plot(t, betha_cc)

In [ ]:
# Módulo de elasticidade

for i in range(3):
    betha_cc = np.exp(s[i]*(1.0-np.sqrt(28.0/t)))
    betha_ce = np.power(betha_cc, 0.3)
    plt.plot(t, betha_ce)